# Bulk inserting segment Ids
This notebook handles the translation of latitude/longitude columns within a dataset to their nearest segment ID. This is a very compute-intensive operation which should only need to be run one time on any given dataset.

In [1]:
import pandas as pd
import requests
from zipfile import ZipFile as zzip
import fiona
import geopandas as gpd
from shapely.geometry import Point
import json

In [2]:
LION_ZIP_DIR = "input_data/nyclion_19b.zip"
GDB_FILE = r"input_data/lion/lion.gdb"
OUTPUT_DIR = "output/"

## Download LION data

In [3]:
# Download and store lion files
url = r"https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyclion_19b.zip"

# download the file contents in binary format
r = requests.get(url)
# open method to open a file on your system and write the contents
with open(LION_ZIP_DIR, "wb") as file:
    file.write(r.content)

# opening the zip file in READ mode
with zzip(LION_ZIP_DIR, 'r') as file:
    # printing all the contents of the zip file
    file.printdir()

    # extracting all the files
    file.extractall("input_data/")
    print('Done!')

File Name                                             Modified             Size
lion/lion.gdb/a00000001.freelist               2019-05-13 12:48:56         4440
lion/lion.gdb/a00000001.gdbindexes             2019-05-13 12:42:56          110
lion/lion.gdb/a00000001.gdbtable               2019-05-13 12:48:56          385
lion/lion.gdb/a00000001.gdbtablx               2019-05-13 12:48:56         5152
lion/lion.gdb/a00000001.TablesByName.atx       2019-05-13 12:48:56         4118
lion/lion.gdb/a00000002.gdbtable               2019-05-13 12:42:56         2055
lion/lion.gdb/a00000002.gdbtablx               2019-05-13 12:42:56         5152
lion/lion.gdb/a00000003.gdbindexes             2019-05-13 12:42:56           42
lion/lion.gdb/a00000003.gdbtable               2019-05-13 12:44:22         1825
lion/lion.gdb/a00000003.gdbtablx               2019-05-13 12:44:22         5152
lion/lion.gdb/a00000004.CatItemsByPhysicalName.atx 2019-05-13 12:48:56         4118
lion/lion.gdb/a00000004.CatItemsByTy

In [4]:
layers = fiona.listlayers(GDB_FILE)
print(layers)

['node', 'node_stname', 'altnames', 'lion']


In [5]:
lion_gdf = gpd.read_file(GDB_FILE, engine='pyogrio', layer='lion')

In [6]:
# Remove rows outside of manhattan just to clean up and speed up operations
print("Before: ", len(lion_gdf.index))
lion_gdf = lion_gdf[lion_gdf.LBoro == 1]
print("After: ", len(lion_gdf.index))

Before:  226977
After:  32966


In [7]:
def get_segment_id_from_coords(lat, lng):
    point = Point(lng, lat)
    if lion_gdf.crs.is_geographic:
        point_gdf = gpd.GeoDataFrame([{'geometry': point}], crs=lion_gdf.crs)
    else:
        point_gdf = gpd.GeoDataFrame([{'geometry': point}], crs="EPSG:4326").to_crs(lion_gdf.crs)
    
    lion_gdf['distance'] = lion_gdf.geometry.distance(point_gdf.iloc[0].geometry)
    nearest_segment = lion_gdf.loc[lion_gdf['distance'].idxmin()]
    segment_id = nearest_segment['SegmentID']
    return segment_id

In [8]:
# Quick test
get_segment_id_from_coords(40.748433, -73.985656)

'0297696'

In [9]:
def load_segment_ids():
    new_segment_ids = {}
    # Opening JSON file
    f = open(f"{OUTPUT_DIR}/segment_id_dict.json")
    data = json.load(f)
    for key in data.keys():
        coordinates_tuple = tuple(map(float, key.split(',')))
        new_segment_ids[coordinates_tuple] = data[key]

    return new_segment_ids

In [10]:
segment_ids = load_segment_ids()

In [11]:
print(list(segment_ids.keys())[:4])
print(list(segment_ids.values())[:4])

[(40.74307, -73.98426), (40.799446, -73.968376), (40.79502, -73.94425), (40.762455, -73.985985)]
['0033181', '0038133', '0038090', '0034097']


In [12]:
def add_segment_id_column(df):
    global segment_ids
    updated_coordinates = []
    for idx, row in df.iterrows():
        try:
            lat = row['Latitude']
            lng = row['Longitude']
            if (lat, lng) not in updated_coordinates: # Need to update this column
                if (lat, lng) not in segment_ids.keys(): # Need to calculate
                    print(f'Calculating segment for ({lat}, {lng})')
                    segment_id = get_segment_id_from_coords(lat, lng)
                    segment_ids[(lat, lng)] = segment_id
                else: # We can pull from our dict
                    segment_id = segment_ids[(lat, lng)]
                matching_rows = df[(df.Latitude == lat) & (df.Longitude == lng)]
                print(f'Updating: {len(matching_rows)}')
                df.loc[(df.Latitude == lat) & (df.Longitude == lng), 'SegmentId'] = segment_id
                
                rows_left = df['SegmentId'].isnull().sum()
                print(f'Rows left: {rows_left}')
                updated_coordinates.append((lat, lng))

                if rows_left == 0:
                    return
            else:
                # Any seg
                pass
        except:
            print(f"Unable to translate ({lat}, {lng}), skipping")

In [15]:
SUBWAY_DATA = "input_data/MTA_Subway_Hourly_Ridership_20240607.csv"
BIKE_DATA_1 = "input_data/202401-citibike-tripdata_1.csv"
BIKE_DATA_2 = "input_data/202401-citibike-tripdata_2.csv"
SUBWAY_DATA_FEB = "input_data/Subway_Transit_Data_FEB_2024.csv"

In [16]:
df_transit = pd.read_csv(SUBWAY_DATA_FEB)

In [17]:
df_transit.columns
df_transit = df_transit[df_transit.borough == 'Manhattan']
df_transit.rename(columns={'latitude': 'Latitude', 'longitude': 'Longitude'}, inplace=True)
df_transit['SegmentId'] = None

df_transit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 632563 entries, 0 to 632562
Data columns (total 15 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   transit_timestamp         632563 non-null  object 
 1   transit_mode              632563 non-null  object 
 2   station_complex_id        632563 non-null  object 
 3   station_complex           632563 non-null  object 
 4   borough                   632563 non-null  object 
 5   payment_method            632563 non-null  object 
 6   fare_class_category       632563 non-null  object 
 7   ridership                 632563 non-null  int64  
 8   transfers                 632563 non-null  int64  
 9   Latitude                  632563 non-null  float64
 10  Longitude                 632563 non-null  float64
 11  Georeference              632563 non-null  object 
 12  Counties                  632563 non-null  int64  
 13  NYS Municipal Boundaries  632563 non-null  i

In [18]:
uniq = df_transit.drop_duplicates(subset=['Latitude', 'Longitude'])
print("Expected length of segment ids: ", len(uniq))
print("Actual: ", len(segment_ids))

Expected length of segment ids:  161
Actual:  6635


In [19]:
add_segment_id_column(df_transit)

Updating: 4735
Rows left: 627828
Updating: 4644
Rows left: 623184
Updating: 4681
Rows left: 618503
Updating: 5134
Rows left: 613369
Updating: 5242
Rows left: 608127
Updating: 5260
Rows left: 602867
Updating: 5653
Rows left: 597214
Updating: 5043
Rows left: 592171
Updating: 4723
Rows left: 587448
Updating: 5404
Rows left: 582044
Updating: 5466
Rows left: 576578
Updating: 4763
Rows left: 571815
Updating: 5248
Rows left: 566567
Updating: 4900
Rows left: 561667
Updating: 5562
Rows left: 556105
Updating: 5217
Rows left: 550888
Updating: 4878
Rows left: 546010
Updating: 5284
Rows left: 540726
Updating: 5061
Rows left: 535665
Updating: 5485
Rows left: 530180
Updating: 5396
Rows left: 524784
Updating: 5205
Rows left: 519579
Updating: 5112
Rows left: 514467
Updating: 5051
Rows left: 509416
Updating: 4782
Rows left: 504634
Updating: 5039
Rows left: 499595
Updating: 4857
Rows left: 494738
Updating: 5347
Rows left: 489391
Updating: 5387
Rows left: 484004
Updating: 5427
Rows left: 478577
Updating: 

In [20]:
df_transit.head()

,transit_timestamp,transit_mode,station_complex_id,station_complex,borough,payment_method,fare_class_category,ridership,transfers,Latitude,Longitude,Georeference,Counties,NYS Municipal Boundaries,SegmentId
0,02/01/2024 12:00:00 AM,subway,304,145 St (1),Manhattan,metrocard,Metrocard - Other,3,0,40.826550,-73.95036,POINT (-73.95036 40.82655),2095,749,0038935
1,02/01/2024 12:00:00 AM,subway,300,191 St (1),Manhattan,omny,OMNY - Full Fare,3,0,40.855225,-73.92941,POINT (-73.92941 40.855225),2095,749,0072489
2,02/01/2024 12:00:00 AM,subway,156,"103 St (C,B)",Manhattan,metrocard,Metrocard - Full Fare,1,0,40.796093,-73.96146,POINT (-73.96146 40.796093),2095,749,0309974
3,02/01/2024 12:00:00 AM,subway,399,68 St-Hunter College (6),Manhattan,omny,OMNY - Full Fare,21,2,40.768143,-73.96387,POINT (-73.96387 40.768143),2095,749,0036431
4,02/01/2024 12:00:00 AM,subway,324,Houston St (1),Manhattan,omny,OMNY - Seniors & Disability,1,0,40.728252,-74.00536,POINT (-74.00536 40.728252),2095,749,0031922


In [21]:
df_transit['transit_timestamp'] = pd.to_datetime(df_transit['transit_timestamp'])
min_start_date = df_transit['transit_timestamp'].min()
max_start_date = df_transit['transit_timestamp'].max()

print(f"Minimum start date: {min_start_date}")
print(f"Maximum start date: {max_start_date}")

Minimum start date: 2024-02-01 00:00:00
Maximum start date: 2024-02-29 23:00:00


/var/folders/k2/s9j0wvg542n0d4_0n5lwnp140000gn/T/ipykernel_20354/820701446.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_transit['transit_timestamp'] = pd.to_datetime(df_transit['transit_timestamp'])


## Add column to bike data

In [22]:
BIKE_DATA_FEB = "input_data/Bike_Data_FEB_2024.csv"
BIKE_DATA_MAR = "input_data/Bike_Data_MAR_2024.csv"

In [23]:
df_bike_1 = pd.read_csv(BIKE_DATA_FEB, dtype={"start_station_id": str})
df_bike_2 = pd.read_csv(BIKE_DATA_MAR, dtype={"start_station_id": str, "end_station_id": str})

FileNotFoundError: [Errno 2] No such file or directory: 'input_data/Bike_Date_FEB_2024.csv'

In [72]:
df_bike = pd.concat([df_bike_1, df_bike_2], ignore_index=True)

In [73]:
df_bike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1954376 entries, 0 to 1954375
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 193.8+ MB


In [74]:
df_bike['started_at'] = pd.to_datetime(df_bike['started_at'])
min_start_date = df_bike['started_at'].min()
max_start_date = df_bike['started_at'].max()

print(f"Minimum start date: {min_start_date}")
print(f"Maximum start date: {max_start_date}")

Minimum start date: 2024-01-01 00:00:03
Maximum start date: 2024-01-31 23:59:59


In [77]:
df_bike_new = pd.DataFrame()
df_bike['started_at'] = pd.to_datetime(df_bike['started_at'])
df_bike['ended_at'] = pd.to_datetime(df_bike['ended_at'])

df_bike_new['Timestamp'] = pd.concat([df_bike['started_at'], df_bike['ended_at']], ignore_index=True)
df_bike_new['Timestamp'] = pd.to_datetime(df_bike_new['Timestamp'])
df_bike_new['Timestamp_Rounded'] = df_bike_new['Timestamp'].dt.round("h")
df_bike_new['Latitude'] = pd.concat([df_bike['start_lat'], df_bike['end_lat']], ignore_index=True)
df_bike_new['Longitude'] = pd.concat([df_bike['start_lng'], df_bike['end_lng']], ignore_index=True)

# Round the lat/lngs
df_bike_new['Latitude'] = df_bike_new['Latitude'].round(3)
df_bike_new['Longitude'] = df_bike_new['Longitude'].round(3)

In [79]:
print("Unique lat/long: ", len(df_bike_new.drop_duplicates(subset=['Latitude', 'Longitude'])))
print("Unique lat/long/time: ", len(df_bike_new.drop_duplicates(subset=['Latitude', 'Longitude', 'Timestamp_Rounded'])))

Unique lat/long:  6475
Unique lat/long/time:  926827


In [81]:
uniq = df_bike_new.drop_duplicates(subset=['Latitude', 'Longitude'])
print("Expected length of segment ids: ", len(segment_ids) + len(uniq))
print("Actual: ", len(segment_ids))

Expected length of segment ids:  6636
Actual:  161


In [84]:
# Split up DF into chunks to be pooled
chunk_size = 100
num_chunks = len(df_bike_new) // chunk_size + (len(df_bike_new) % chunk_size > 0)
chunks = [df_bike_new.iloc[i * chunk_size:(i + 1) * chunk_size] for i in range(num_chunks)]
print(f"Broke up bike into {len(chunks)} chunks")

Broke up bike into 39088 chunks


In [ ]:
add_segment_id_column(df_bike_new)

In [116]:
print("Length of segment ids: ", len(segment_ids))
df_bike_new.head()

Length of segment ids:  6635


,Timestamp,Timestamp_Rounded,Latitude,Longitude,SegmentId
0,2024-01-25 20:39:09,2024-01-25 21:00:00,40.735,-73.991,0032805
1,2024-01-15 18:44:36,2024-01-15 19:00:00,40.735,-73.988,0032949
2,2024-01-03 19:27:58,2024-01-03 19:00:00,40.735,-73.988,0032949
3,2024-01-22 18:29:46,2024-01-22 18:00:00,40.735,-73.988,0032949
4,2024-01-27 09:55:39,2024-01-27 10:00:00,40.735,-73.988,0032949


In [115]:
# Convert segment ids into json format
segment_ids_as_json = {}
for key in segment_ids.keys():
    tuple_str = ",".join([str(key[0]), str(key[1])])
    segment_ids_as_json[tuple_str] = segment_ids[key]

# Save segment ids for future use
with open(f"{OUTPUT_DIR}/segment_id_dict.json", "w") as outfile: 
    json.dump(segment_ids_as_json, outfile)
    print("Segment ids saved")

Segment ids saved


In [118]:
# Save our dataframes
SUBWAY_DATA = "input_data/MTA_Subway_Hourly_Ridership_20240607.csv"
BIKE_DATA_1 = "input_data/202401-citibike-tripdata_1.csv"
BIKE_DATA_2 = "input_data/202401-citibike-tripdata_2.csv"

bike_csv = f"{OUTPUT_DIR}/202401-citibike-tripdata_with_segments.csv"
df_bike_new.to_csv(bike_csv, index=False)
print(f"Bike data saved: {bike_csv}")

Bike data saved: output//202401-citibike-tripdata_with_segments.csv
Transit data saved: output//MTA_Subway_Hourly_Ridership_20240607_with_segments.csv


In [28]:
transit_csv = f"{OUTPUT_DIR}/MTA_Subway_Hourly_Ridership_20240607_with_segments.csv"
df_transit.to_csv(transit_csv, index=False)
print(f"Transit data saved: {transit_csv}")

Transit data saved: output//MTA_Subway_Hourly_Ridership_20240607_with_segments.csv


In [30]:
check_df = pd.read_csv(transit_csv)
check_df.head()

,transit_timestamp,transit_mode,station_complex_id,station_complex,borough,payment_method,fare_class_category,ridership,transfers,Latitude,Longitude,Georeference,Counties,NYS Municipal Boundaries,New York Zip Codes,SegmentId
0,2024-01-01 00:00:00,subway,404,28 St (6),Manhattan,metrocard,Metrocard - Fair Fare,8,0,40.743070,-73.984260,POINT (-73.98426 40.74307),2095,749,739.0,33181
1,2024-01-01 00:00:00,subway,309,103 St (1),Manhattan,metrocard,Metrocard - Fair Fare,3,0,40.799446,-73.968376,POINT (-73.968376 40.799446),2095,749,752.0,38133
2,2024-01-01 00:00:00,subway,394,110 St (6),Manhattan,metrocard,Metrocard - Other,3,0,40.795020,-73.944250,POINT (-73.94425 40.79502),2095,749,756.0,38090
3,2024-01-01 00:00:00,subway,162,"50 St (C,E)",Manhattan,omny,OMNY - Seniors & Disability,1,0,40.762455,-73.985985,POINT (-73.985985 40.762455),2095,749,742.0,34097
4,2024-01-01 00:00:00,subway,403,33 St (6),Manhattan,metrocard,Metrocard - Unlimited 30-Day,9,0,40.746082,-73.982080,POINT (-73.98208 40.746082),2095,749,739.0,281182
